# Custom YOLO Model

I used the YOLOv8 nano model and retrained it to help my program identify the correct links on a webpage to click.

## Model Training
This code snippet shows the process I used to create the custom model, along with the annotated screenshots I prepared using 'labelImg'.

## Model Evaluation
I evaluated the current model and found it to have an impressive 97.7% accuracy. This high accuracy can be attributed to the model's power and the relative simplicity of the task.

## Testing
Before integrating the custom model into my main program, I thoroughly tested it to ensure its performance and reliability.


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load a pretrained model
# Train the model
model.train(data='dataset.yaml', epochs=100, imgsz=640)

metrics = model.val() #eval
# Load an image
image = cv2.imread('test.png')
# Perform link detection
results = model(image)
# Visualise the results
annotated_image = results[0].plot()
cv2.imshow('Link Detection', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

def testing():
    model = YOLO('./runs/detect/train16/weights/best.pt')
    image = cv2.imread('test.png')
    results = model(image)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    cv2.imshow('Detections', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
import pyautogui
import time
import keyboard
import pytesseract
from PIL import  ImageGrab
import pandas as pd
import datetime
import requests
from ultralytics import YOLO
import easyocr
import os

ModuleNotFoundError: No module named 'cv2'

# Project Setup

I begin by initialising various variables and the two models I'll be using for this data analytics project.

## Current Date
I retrieve the current day to add dates to my data later in the project.

## OCR Screen Cropping
I set `left`, `top`, and other values to specify which section of the screen to crop for OCR in the second section.

## OCR Model
For optical character recognition, I use the pre-trained EasyOCR English model.

## Link Detection Model
For link detection, I employ a custom-trained YOLOv8 model, specifically the lightweight nano version (before training).

## Data Mapping
I define a mapping dictionary and a rolls list for data mapping once I've gathered the necessary information.

In [2]:
print("Starting now!")
global current_day 
current_day  = datetime.datetime.now().day
#pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

left = 846
top = 506
right = 1073
bottom = 666
messages_already_send = []

# Initialise the YOLOY v8 custom model
model = YOLO('./best_link_model/best.pt')

# Initialise the EasyOCR reader
reader = easyocr.Reader(['en'], gpu = True)  # Specify the language(s) you want to recognize

stat_mapping = {
        'melee damage': 'md',        'multishot': 'ms',        'fire rate': 'fr',        'attack speed': 'as',        'damage to corpus': 'corp',
        'damage to grineer': 'grin',        'damage to infested': 'inf',        'mpact': 'imp',        'puncture': 'pun',        'slash': 'slash',
        'cold': 'cold',        'electric': 'ele',        'heat': 'heat',        'toxin': 'tox',        'combo duration': 'combo',
        'critical chance': 'cc',        'critical damage': 'cd',        'slide crit': 'slide',        'finisher damage': 'fin',
        'projectile speed': 'pfs',        'ammo': 'ammo',        'magazine': 'mag',        'punch through': 'pt',        'reload speed': 'reload',
        'range': 'rng',        'status chance': 'sc',        'status duration': 'sd',        'weapon recoil': 'rec',        'zoom': 'z',
        'initial combo': 'ic',        'heavy attack': 'eff',        'combo chance': 'combogain'
    }

# this is a reference list for the in game items I want the program to flag
perfect_rolls = [    ['ic', 'cc', 'cd'],    ['cc', 'cd', 'md'],    ['cc', 'cd', 'ms'],    ['cc', 'cd', 'as'],    ['cd', 'rng', 'as']]

Starting now!


NameError: name 'datetime' is not defined

# OCR Data Processing

This function processes the main screenshot after applying EasyOCR and uses logic to add the extracted data to my DataFrame if it meets certain criteria.

## Data Mapping
I leverage the consistent order of data and specific string key characters (such as '+' and `isdigit()`) to assign the OCR list of strings to the correct column in my DataFrame.

## Data Filtering
To avoid massive, unreadable DataFrames and CSVs, I filter out all the less important rows, focusing on the essential information.

In [3]:
def format_input(df, input_list,username_value,time_value,tradetext_value):
    full = False
    #close function when false positive happens (1/50 of the time)
    if len(input_list) < 3:
        return(df)
        
    if len(input_list[0].split()) == 3:
        weapon = ' '.join(input_list[0].split()[:2])
    else:
        try:
            weapon = input_list[0].split()[0]
        except IndexError:
            weapon = 'index error'  

    # Extract name
    if not input_list[1].startswith('+'):
        name = input_list[0].split(weapon)[-1].strip() + ' ' + input_list[1]
    else:
        name = input_list[0].split(weapon)[-1].strip()

    # Initialise stat variables
    stat_1, stat_1_value, stat_2, stat_2_value, stat_3, stat_3_value, stat_4, stat_4_value = '', '', '', '', '', '', '', ''

    # Extract stat information
    for i, item in enumerate(input_list):
        if '(x2' not in item:           
            if any(char.isdigit() for char in item):
                if not stat_1:
                        stat_1 = ' '.join(item.split()[1:]).lower()
                        if not stat_1:
                            try:
                                stat_1 = input_list[i+1]
                            except:
                                stat_1 = None
                        stat_1_value = item.split()[0]
                elif not stat_2:

                        stat_2 = ' '.join(item.split()[1:]).lower()
                        if not stat_2:
                            try:
                                stat_2 = input_list[i+1]
                            except:
                                stat_2 = None
                        stat_2_value = item.split()[0]
                elif not stat_3:

                        stat_3 = ' '.join(item.split()[1:]).lower()
                        if not stat_3:
                            try:
                                stat_3 = input_list[i+1]
                            except:
                                stat_3 = None
                        stat_3_value = item.split()[0]
                elif not stat_4 and any(char.isdigit() for char in item):
                    full = True
                    stat_4_parts = item.split()
                    stat_4_value = stat_4_parts[0]
                    if stat_4_parts[1:]:
                        stat_4 = ' '.join(stat_4_parts[1:])
                    else:
                        stat_4 = input_list[i+1]
    if full == False:
        return(df)
    # Create a new row as a dictionary
    new_row = {
        'weapon': weapon,        'name': name,        'username':username_value,        'time':time_value,        'full': full,        'stat_1': stat_1,
        'stat_1_value': stat_1_value,        'stat_2': stat_2,        'stat_2_value': stat_2_value,        'stat_3': stat_3,
        'stat_3_value': stat_3_value,        'stat_4': stat_4,        'stat_4_value': stat_4_value,        'tradetext':tradetext_value
    }
    # Map stat values to aliases
    roll_parts = []
    for i in range(1, 5):
        stat_key = f'stat_{i}'
        if new_row[stat_key].strip():
            stat_alias = ''
            for stat, alias in stat_mapping.items():
                if stat in new_row[stat_key].strip().lower():
                    stat_alias = alias
                    break
            if i == 4:
                stat_alias = '-' + stat_alias
            roll_parts.append(stat_alias)

    new_row['roll'] = ','.join(roll_parts)

    new_row['perfect'] = False
    if any(set(new_row['roll'].split(',')[:3]) == set(perfect_roll) for perfect_roll in perfect_rolls):
        new_row['perfect'] = True
    # Convert the row data to a DataFrame
    new_row = pd.DataFrame([new_row])

    # Concatenate the new row with the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    return df

# Screenshot Processing

This function captures a screenshot of the screen, crops it to the desired region, and then converts the cropped image into a NumPy array. I use this format because it is the input format required by EasyOCR for further processing.


In [4]:
def ocr_image_easyocr(left, top, right, bottom):
    # Capture the specified region of the screen
    screenshot = ImageGrab.grab(bbox=(left, top, right, bottom))
    screenshot = screenshot.convert('RGB')
    # Convert the PIL image to a numpy array
    screenshot_np = np.array(screenshot)
    
    return screenshot_np

# OCR Processing

This function performs optical character recognition (OCR) on the NumPy array, converting it into a list of strings.

## Optimisation
I split up the screenshot capture and OCR processes to optimise my program further. By doing this, I can perform OCR when my program needs to sleep anyway due to webpage loading, making efficient use of the available time.


In [5]:
def easyocr_process(screenshot_np):
    results = reader.readtext(screenshot_np, detail = 0)
    return results

In [ ]:
def extract_name_and_time(line_top, line_bottom):
    
    name_value = None
    time_value = None
    for i in range(2):
        line_left = 515
        line_right = 860

        line_image = ocr_image_easyocr(line_left, line_top, line_right, line_bottom)
        line_text = easyocr_process(line_image)
        if i > 0:
            print(f"moved upwards new trade text: {line_text}")
        if len(line_text) > 0:
            
            line_text = line_text[0]
        if '.' in line_text:
            # Extract the time and name values from the line text
            time_start = line_text.find('[') + 1
            time_end = line_text.find(']')
            time_value = line_text[time_start:time_end]

            name_start = time_end + 2
            name_value = line_text[name_start:].split(':')[0].strip()
            time_value = f"{current_day} {time_value}"
            break
        else:
            line_top -= 32
            line_bottom-= 32
            print(f"Couldnt find '.': {line_text}")
            
    if name_value is None:
        print(f"No name found on the line with the link: {line_text}")
    return name_value, time_value,line_text

# Main Function

This is my main function that runs the entire program. Its primary responsibility is handling the YOLO model, which is used to identify links on the page within a specific area and then loop through those links.

## Link Selection and OCR
Each time a link is selected, the function calls other functions to perform OCR on the visible page as well as the page that appears when the link is clicked.

## PyAutoGUI
I use PyAutoGUI to control the mouse and keyboard for automation. I opted for non-standard functions as I found them to be more reliable when some automated clicks are suppressed.

## Operation Ordering
The order of operations might seem slightly odd, but it's designed to perform process-intensive tasks like EasyOCR and YOLO models when the program needs to wait for the page to load anyway.

## Time Values and Screenshot Processing
The presence of `current` and `regular` time values, as well as the last screenshot processing, is a result of this ordering.

## Link Coordinates
The y-coordinates of the links are used for OCR selection on the first page.


In [ ]:
def click_links(df,model):
    # Set the region of interest (ROI) coordinates
    roi_left = 515
    roi_top = 173
    roi_right = 1822
    roi_bottom = 953
    screenshot_np = None  

    while True:
        screenshot = pyautogui.screenshot(region=(roi_left, roi_top, roi_right - roi_left, roi_bottom - roi_top))
        screenshot = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

        # Perform link detection using the YOLOv8 model
        results = model(screenshot)

        # Iterate over the detected links
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Extract the link position from the bounding box
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                link_center = (int((x1 + x2) / 2) + roi_left, int((y1 + y2) / 2) + roi_top)
                link_y = link_center[1]
                #area of the page when to bound box of the link is found
                line_top = link_y - 16
                line_bottom = link_y + 16
                    
                pyautogui.moveTo(link_center[0], link_center[1])
                current_name_value, current_time_value,current_line_text = extract_name_and_time(line_top, line_bottom)
                time.sleep(0.2)
                pyautogui.mouseDown(button='left')
                pyautogui.mouseUp(button='left')

                time.sleep(0.6)
                #last loop image processing
                if screenshot_np is not None:
                    input_list = easyocr_process(screenshot_np)
                    df = format_input(df, input_list,name_value,time_value,line_text)
                else:
                    time.sleep(0.3)
 
                time_value = current_name_value
                name_value = current_name_value
                line_text = current_line_text

                screenshot_np = ocr_image_easyocr(left, top, right, bottom)

                # Press the escape key if the screen has changed significantly
                keyboard.send('esc')
                
                time.sleep(0.2)
                keyboard.send('t')
                time.sleep(0.2)
        #processing last screenshot before breaking
        input_list = easyocr_process(screenshot_np)
        df = format_input(df, input_list,name_value,time_value,line_text)
        # Break the loop if all links have been clicked
        break

    return df

This functuon uses webhooks this sends a discord message when a link is flagged

In [ ]:
def send_discord_message(message):
    payload = {
        "content": message
    }
    response = requests.post('xxxxxxxxxxxxx', json=payload)
    if response.status_code == 204:
        print(message)
        
    else:
        print(f"Failed to send message. Status code: {response.status_code}")

In [ ]:
#Setting up dataframe
df = pd.DataFrame(columns=['weapon','name','username','perfect','roll', 'full', 'stat_1', 'stat_1_value', 'stat_2', 'stat_2_value',
                           'stat_3', 'stat_3_value', 'stat_4', 'stat_4_value'])

# Main Code

The main code continuously loops over new data, identifying key links and sending Discord messages containing their DataFrame information.

## Data Persistence
It saves a CSV file of important links as well as all links encountered during the process.



In [ ]:
while True:
    time.sleep(0.2)
    pyautogui.scroll(-5000) 
    time.sleep(0.2)
    pyautogui.scroll(400)
    df = click_links(df,model)

    df = df.drop_duplicates(subset=['weapon', 'stat_1_value'], keep='first')
    columnstodrop = ['tradetext','full']
    
    df['id'] = df['stat_1_value'].astype(str) + df['stat_2_value'].astype(str) + df['stat_3_value'].astype(str) + df['stat_4_value'].astype(str)
    
    dfshort = df[df['perfect']==True]
    dfshort = dfshort[['weapon','roll','username','time','id']]
    print(dfshort.to_string(index=False))
    #checking for important rows and sending discord message
    for index, row in df.iterrows():
        if row['perfect'] == True:
            message = f"Weapon: {row['weapon']}\nRoll: {row['roll']}\nUsername:{row['username']}"
            rivenid = row['weapon'] + str(row['stat_1_value'])
            username = row['username']
            if rivenid not in messages_already_send and username not in messages_already_send:
                send_discord_message(message)
                messages_already_send.append(rivenid)
                messages_already_send.append(username)
                
    dfshort.to_csv('short-trade.csv', mode='a', na_rep='N/A', header=False, index=False, escapechar="\n")
    df.to_csv('trade.csv', mode='a', na_rep='N/A', header=False, index=False, escapechar="\n")
    time.sleep(60)

This is a function I used for identifing screen coordinates from screenshots using matplotlib

In [ ]:
import cv2
import matplotlib.pyplot as plt

screenshot = cv2.imread('test.png')

def mouse_callback(event):
    if event.button == 1:  # Left mouse button
        x, y = event.xdata, event.ydata
        print(f"Clicked coordinates: ({int(x)}, {int(y)})")

plt.imshow(cv2.cvtColor(screenshot, cv2.COLOR_BGR2RGB))
plt.connect('button_press_event', mouse_callback)
plt.show()